# Normal ANN for observation operator

## Load in the required data

In [ ]:
import os
import pandas as pd
from pathlib import Path
pad = Path(os.getcwd())
if pad.name == "ml_observation_operator":
    pad_correct = pad.parent
    os.chdir(pad_correct)
%run "ml_observation_operator/data_load_in.py"

Load in

In [ ]:
ML_data_pad = Path("data/Zwalm_data/ML_data")
X_train_all = pd.read_pickle(ML_data_pad/"X_train.pkl")
X_test_all = pd.read_pickle(ML_data_pad/"X_test.pkl")
X_full_all = pd.read_pickle(ML_data_pad/"X_full.pkl")

X_train = pd.read_pickle(ML_data_pad/"X_train.pkl")
X_test = pd.read_pickle(ML_data_pad/"X_test.pkl")
X_full = pd.read_pickle(ML_data_pad/"X_full.pkl")

X_train_small = pd.read_pickle(ML_data_pad/"X_train_small.pkl")
X_test_small = pd.read_pickle(ML_data_pad/"X_test_small.pkl")
X_full_small = pd.read_pickle(ML_data_pad/"X_full_small.pkl")

y_train = pd.read_pickle(ML_data_pad/"y_train.pkl")
y_test = pd.read_pickle(ML_data_pad/"y_test.pkl")
y_full = pd.read_pickle(ML_data_pad/"y_full.pkl")

Cstar = pd.read_pickle(ML_data_pad/"Cstar.pkl")

Drop deltat_t! not needed since 1 timestep at a time considered!

In [ ]:
X_train = X_train.drop('delta_t',axis = 1)
X_test = X_test.drop('delta_t',axis = 1)
X_full = X_full.drop('delta_t',axis = 1)
display(X_full.head())

X_train_small = X_train_small.drop('delta_t',axis = 1)
X_test_small = X_test_small.drop('delta_t',axis = 1)
X_full_small = X_full_small.drop('delta_t',axis = 1)
display(X_full_small.head())

## Load in used packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import dask
from dask.distributed import Client
import pickle
import itertools
import random as python_random
import tensorflow as tf
from tensorflow import keras
from keras import (models,layers)
from sklearn.metrics import r2_score
 #experiment
from sklearn.neural_network import MLPRegressor
from functions.plotting_functions import ensemble_plot
from functions.ml_utils import (general_tensorflow_model, validation_loop,
                                full_training_loop)
SEED =1234
exec_training = False
#os.environ['PYTHONHASHSEED'] = str(SEED)
#os.environ['TF_DETERMINISTIC_OPS'] = '1'
python_random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
%load_ext autoreload 
%autoreload 2 

Check if GPU present

In [ ]:
tf.config.list_physical_devices('GPU')

Experiment: load in dask, tip to set proccesse to False seems to stop kernel from crashing! (gotten from https://e-marshall.github.io/sentinel1_rtc/asf_local_mf.html, who also had crashing kernels!)

In [ ]:
client = Client(processes = False)
client

In [ ]:
#client.close()

# Recreate Linear Regression in tensorflow

https://stackoverflow.com/questions/75759597/typeerror-isinstance-arg-2-must-be-a-type-or-tuple-of-types-in-tensorflow-add why typeguard specification is needed

In [ ]:
n_features = X_full.shape[1]
print("Number of features: " + str(n_features))
lin_reg = models.Sequential(
    [
    layers.Input(shape = (n_features,)),
    #layers.Normalization(),
    layers.Dense(1, activation = "linear", name = 'layer1'), #first number = dimensionality of the output space!
    ]
)
lin_reg.summary()
epochs = 200
out_dict = general_tensorflow_model(lin_reg, X_train.values, X_test.values, y_train.values,
                                     y_test.values, X_train.index, X_test.index, Cstar,
                                    epochs = epochs, validation_split= 0)


Adding the repeats: first determine optimal number of epochs on 20% hold out

In [ ]:
n_train = X_train.shape[0]
print(n_train)

In [ ]:
repeats = 8
out_list = []
#For later storage
pad = Path('data/ml_obs_op_data/ann/linear_regression')
if not os.path.exists(pad):
    os.makedirs(pad)
mean_r2_val, recommended_nr_epochs = validation_loop(
    lin_reg, repeats, X_train, X_test, y_train, y_test, Cstar,
    exec_training, pad, epochs = epochs, dask_bool = True,
    print_output = False, verbose = 0, validation_split = 0.2
)

Repeat for optimum number of epochs on full training dataste

In [ ]:
mean_r2_train, mean_r2_test, out_dict, models_list = full_training_loop(
 lin_reg, repeats,X_train, X_test,y_train,y_test, Cstar,
 exec_training, recommended_nr_epochs, pad, print_output = False,
 verbose = 0, validation_split = 0.0, dask_bool = False
)                                        

In [ ]:
ensemble_plot(Cstar, X_full.index, out_dict)

# Simple ANN model

https://github.com/keras-team/keras/issues/597 validation_split argument in `.fit()` does not shuffle the dataset! 

## Full dataset

Following the recommendataions as given in [understanding-deep-learning-chitta-ranjan-2021b](https://drive.google.com/file/d/1fejSMGPIDMO4eilsIDj-7QcFsCge4vdv/view)

use 2 hidden layers as default
- 1st layer: #nodes = 1/2 of of input nodes 
- 2nd layer: #nodes = 1/2 of number of nodes in previous layer

relu as default activation and no dropout


In [ ]:
#exec_training = True
ann = models.Sequential(
    [
    layers.Input(shape = (n_features,)),
    layers.Dense(6, activation = "relu", name = 'layer1'), #first number = dimensionality of the output space!
    layers.Dense(3, activation = "relu", name = 'layer2'),
    layers.Dense(1, activation = "linear", name = 'layer3'),
    ]
)

ann.summary()

In [ ]:
repeats = 8
epochs = 100
out_list = []
#For later storage
pad = Path('data/ml_obs_op_data/ann/simple_ann')
if not os.path.exists(pad):
    os.makedirs(pad)
mean_r2_val, recommended_nr_epochs = validation_loop(
        ann, repeats, X_train, X_test,y_train, y_test, Cstar,
        exec_training,pad,epochs = epochs,print_output = False,
        verbose = 0, validation_split = 0.2, dask_bool = False
)

In [ ]:
print(mean_r2_val);print(recommended_nr_epochs)

Repeat for the optimal number of epochs on full training data

In [ ]:
mean_r2_train, mean_r2_test, out_dict, models_list = full_training_loop(
 ann, repeats,X_train, X_test,y_train,y_test, Cstar,
 exec_training, recommended_nr_epochs, pad, print_output = False,
 verbose = 0, validation_split=0.0, dask_bool = False
)

In [ ]:
fig, ax = ensemble_plot(Cstar, X_full.index, out_dict)
ax.legend(loc = 'lower left')
figpad = Path('Figures/Figures_chapter_ML_obs_op')
if not os.path.exists(figpad):
    os.makedirs(figpad)
fig.savefig(figpad/'ANN_small_obs.pdf', format = 'pdf')

Lower than linear regression :/ 

In [ ]:
print('test')
print(out_dict['r2_test_list'])
print(np.min(out_dict['r2_test_list']))
print(np.max(out_dict['r2_test_list']))

print('train')
print(out_dict['r2_train_list'])
print(np.min(out_dict['r2_train_list']))
print(np.max(out_dict['r2_train_list']))

### Experiment with scitkitlearn

In [ ]:
ann_sklearn = MLPRegressor(hidden_layer_sizes=(2,1),solver='lbfgs',random_state = SEED)
from functions.ml_utils import general_sklearn_model
general_sklearn_model(ann_sklearn, X_train.values, X_test.values, y_train.values.reshape(-1,1),
                       y_test.values.reshape(-1,1), X_train.index, X_test.index, Cstar, normalisation=True)

So clearly a much better optimizer for the current state!

## Limited dataset

In [ ]:
repeats = 8
epochs = 100
#For later storage
pad = Path('data/ml_obs_op_data/ann/simple_ann_small_data')
if not os.path.exists(pad):
    os.makedirs(pad)
n_features = X_train_small.shape[1]
ann_small = models.Sequential(
    [
    layers.Input(shape = (n_features,)),
    layers.Dense(6, activation = "relu", name = 'layer1'), #first number = dimensionality of the output space!
    layers.Dense(3, activation = "relu", name = 'layer2'),
    layers.Dense(1, activation = "linear", name = 'layer3'),
    ]
)
ann_small.summary()

In [ ]:
mean_r2_val, recommended_nr_epochs = validation_loop(
        ann_small, repeats, X_train_small, X_test_small,y_train, y_test, Cstar,
        exec_training,pad,epochs = epochs,print_output = False, dask_bool = False,
        verbose = 0, validation_split = 0.2)

In [ ]:
print(mean_r2_val); print(recommended_nr_epochs)

In [ ]:
mean_r2_train, mean_r2_test, out_dict, models_list = full_training_loop(
 ann_small, repeats,X_train_small, X_test_small,y_train,y_test, Cstar,
 exec_training, recommended_nr_epochs, pad, print_output = False,
 verbose = 0, validation_split=0.0, dask_bool = False
)

In [ ]:
n_std = 1
ensemble_plot(Cstar, X_full.index, out_dict)

# More extended MLP model

## Full dataset

Recommendation of Hans: 2n+1 as number of nodes
Add some dropout to prevent overfitting (0.2)

In [ ]:
n_features = X_train.shape[1]
print(2*n_features+1) #round of to 24 for first layer
ann_extended = models.Sequential(
    [
    layers.Input(shape = (n_features,)),
    layers.Dense(24, activation = "relu", name = 'layer1'), #first number = dimensionality of the output space!
    layers.Dropout(0.2),
    layers.Dense(12, activation = "relu", name = 'layer2'),
    layers.Dropout(0.2),    
    layers.Dense(1, activation = "linear", name = 'layer3'),
    ]
)
pad = Path('data/ml_obs_op_data/ann/extende_ann')
if not os.path.exists(pad):
    os.makedirs(pad)
ann_extended.summary()

In [ ]:
mean_r2_val, recommended_nr_epochs = validation_loop(
        ann_extended, repeats, X_train, X_test,y_train, y_test, Cstar,
        exec_training,pad,epochs = epochs,print_output = False,
        verbose = 0, validation_split = 0.2, dask_bool = False)
print(mean_r2_val); print(recommended_nr_epochs)

In [ ]:
mean_r2_train, mean_r2_test, out_dict, models_list = full_training_loop(
 ann_extended, repeats,X_train, X_test,y_train,y_test, Cstar,
 exec_training, recommended_nr_epochs, pad, print_output = False,
 verbose = 0, validation_split=0.0
)

In [ ]:
n_std = 1
ensemble_plot(Cstar, X_full.index, out_dict)

In [ ]:
print('test')
print(out_dict['r2_test_list'])
print(np.min(out_dict['r2_test_list']))
print(np.max(out_dict['r2_test_list']))

print('train')
print(out_dict['r2_train_list'])
print(np.min(out_dict['r2_train_list']))
print(np.max(out_dict['r2_train_list']))

In [ ]:
# ann_sklearn = MLPRegressor(hidden_layer_sizes=(32,16),solver='lbfgs')
# general_sklearn_model(ann_sklearn, X_train.values, X_test.values, y_train.values.reshape(-1,1),
#                        y_test.values.reshape(-1,1), X_train.index, X_test.index, Cstar, normalisation=True)

## Limited dataset

In [ ]:
n_features_small  =  X_train_small.shape[1]
print(n_features_small*2+1)
ann_extended_small = models.Sequential(
    [
    layers.Input(shape = (n_features_small,)),
    layers.Dense(12, activation = "relu", name = 'layer1'), #first number = dimensionality of the output space!
    layers.Dense(6, activation = "relu", name = 'layer2'),
    layers.Dense(1, activation = "linear", name = 'layer3'),
    ]
)
pad = Path('data/ml_obs_op_data/ann/extended_ann_small_data')
if not os.path.exists(pad):
    os.makedirs(pad)
ann_extended.summary()

In [ ]:
ann_extended_small.summary()

In [ ]:
mean_r2_val, recommended_nr_epochs = validation_loop(
        ann_extended_small, repeats, X_train_small, X_test_small,y_train, y_test, Cstar,
        exec_training,pad,epochs = epochs,print_output = True,
        verbose = 0, validation_split = 0.2)

In [ ]:
mean_r2_train, mean_r2_test, out_dict, models_list = full_training_loop(
 ann_extended_small, repeats,X_train_small, X_test_small,y_train,y_test, Cstar,
 exec_training, recommended_nr_epochs, pad, print_output = False,
 verbose = 0, validation_split=0.0
)

In [ ]:
ensemble_plot(Cstar, X_full.index, out_dict)

# A more complex model structure

Not repeated here, was not a great experiment... see OLD notebook

# Hyperparameter tuning

Executed analogous to that of LSTM. Since now no changing input sequence, a 20% validation fraction can be used! Parameters to try are:
- number of layers: 2 or 3
- number of nodes per layer: 4,12,20,28
- dropout rate: 0, 0.2, 0.4 (uniform for all layers and between
each layer)
- large or small dataset

(not checking for time drop or not, limit computation)

Limit to 4 repeats per number of units! 

In [ ]:
pad = Path('data/ml_obs_op_data/ann/hyperparam_tuning')
if not os.path.exists(pad):
    os.makedirs(pad)

In [ ]:
append = True

max_epochs = 100 #max number to try (is the default in general_tensorflow_model)
validation_split = 0.2
repeats = 4 #number of times to repeat training (idea of limiting computational load)
range_nodes = np.arange(4,33,8)
range_nr_layers = [2,3]
# range_forest = [True, False]
# range_time = [True, False]
range_size = ['large','small']
range_learning_rate = [1e-3] #as in Kratzerts
range_dropout_rate = [0,0.2,0.4] 
n_train = X_train.shape[0]
max_nr_options = sum([len(range_nodes)**layer for layer in range_nr_layers])*len(
    range_learning_rate)*len(range_dropout_rate)*len(range_size)
print(f"Total Number of combinations {max_nr_options}") 
col_names = ['data_in_size','nodes_1','nodes_2','nodes_3','learning_rate',
             'drop_out_rate','R2_val','recommended_nr_epochs']
if append:
     pd_hyperparam_val = pd.read_csv(pad/'ann_hyperparam_validation.csv')
else:
    pd_hyperparam_val = pd.DataFrame(columns=col_names, index = range(0,max_nr_options))

In [ ]:
pd_hyperparam_val.head(10)

In [ ]:
dask_bool = False #slower but safer...
iter = 0
if exec_training:
    for size_set in range_size:
        if size_set == 'large':
            X_temp_train = X_train.copy()
            X_temp_test = X_test.copy()
        elif size_set == 'small':
            X_temp_train = X_train_small.copy()
            X_temp_test = X_test_small.copy()
        else:
            raise ValueError('size_set shoulde be large or small')
        n_features = X_temp_train.shape[1]
        for nr_layers in range_nr_layers:
            if nr_layers == 2:
                network_combos = itertools.product(
                    range_nodes, range_nodes, range_dropout_rate,
                    range_learning_rate
                )
                for network_combo in network_combos:
                    print(network_combo)
                    (nodes_1, nodes_2, dropout_rate, 
                     learning_rate) = network_combo
                    ann = models.Sequential(
                        [
                        layers.Input(shape = (n_features,)),
                        layers.Dense(nodes_1, activation = "relu", name = 'layer1'),
                        layers.Dropout(dropout_rate),
                        layers.Dense(nodes_2, activation = "relu", name = 'layer2'),
                        layers.Dropout(dropout_rate),
                        layers.Dense(1, activation = "linear", name = 'layer3'),
                        ]
                    )
                    ann.summary()
                    if append:
                        if pd_hyperparam_val.iloc[iter,:].isna().all():
                            mean_r2_val, recommended_nr_epochs = validation_loop(
                                ann, repeats, X_temp_train, X_temp_test, y_train, y_test,
                                Cstar, exec_training, pad, validation_split= validation_split,
                                learning_rate = learning_rate, verbose = 0, print_output = False,
                                dask_bool = dask_bool
                            )
                            print('validation executed')
                            pd_hyperparam_val.iloc[iter,:] = [size_set, nodes_1, nodes_2, np.nan,
                            learning_rate, dropout_rate, mean_r2_val, recommended_nr_epochs]
                    else:
                        mean_r2_val, recommended_nr_epochs = validation_loop(
                            ann, repeats, X_temp_train, X_temp_test, y_train, y_test,
                            Cstar, exec_training, pad, validation_split= validation_split,
                            verbose = 0, print_output = False, dask_bool = dask_bool
                        )
                        pd_hyperparam_val.iloc[iter,:] = [size_set, nodes_1, nodes_2, np.nan,
                            learning_rate, dropout_rate, mean_r2_val, recommended_nr_epochs]
                    iter = iter  + 1 
                    pd_hyperparam_val.to_csv(pad/'ann_hyperparam_validation.csv', index = False)
                    print(iter)

            elif nr_layers == 3:
                network_combos = itertools.product(
                    range_nodes, range_nodes, range_nodes, range_dropout_rate,
                    range_learning_rate
                )
                for network_combo in network_combos:
                    print(network_combo)
                    (nodes_1, nodes_2, nodes_3, dropout_rate,
                     learning_rate) = network_combo
                    ann = models.Sequential(
                        [
                        layers.Input(shape = (n_features,)),
                        layers.Dense(nodes_1, activation = "relu", name = 'layer1'),
                        layers.Dropout(dropout_rate),
                        layers.Dense(nodes_2, activation = "relu", name = 'layer2'),
                        layers.Dropout(dropout_rate),
                        layers.Dense(nodes_3, activation = "relu", name ='layer3'),
                        layers.Dropout(dropout_rate),
                        layers.Dense(1, activation = "linear", name = 'layer4'),
                        ]
                    )
                    ann.summary()
                    if append:
                        if pd_hyperparam_val.iloc[iter,:].isna().all():
                            mean_r2_val, recommended_nr_epochs = validation_loop(
                                ann, repeats, X_temp_train, X_temp_test, y_train, y_test,
                                Cstar, exec_training, pad, validation_split= validation_split,
                                learning_rate=learning_rate, verbose = 0, print_output = False,
                                dask_bool = dask_bool
                            )
                            pd_hyperparam_val.iloc[iter,:] = [size_set, nodes_1, nodes_2, nodes_3,
                            learning_rate, dropout_rate, mean_r2_val, recommended_nr_epochs]
                    else:
                        mean_r2_val, recommended_nr_epochs = validation_loop(
                            ann, repeats, X_temp_train, X_temp_test, y_train, y_test,
                            Cstar, exec_training, pad, validation_split= validation_split,
                            verbose = 0, print_output = False, dask_bool= dask_bool
                        )
                        pd_hyperparam_val.iloc[iter,:] = [size_set, nodes_1, nodes_2, nodes_3,
                            learning_rate, dropout_rate, mean_r2_val, recommended_nr_epochs]

                    iter = iter  + 1
                    pd_hyperparam_val.to_csv(pad/'ann_hyperparam_validation.csv', index = False)
                    print(iter)
else:
    pd_hyperparam_val = pd.read_csv(pad/'ann_hyperparam_validation.csv')

In [ ]:
pd_hyperparam_val_sorted = pd_hyperparam_val.sort_values('R2_val',ascending=False)
pd_hyperparam_val_sorted.head(10)

In [ ]:
pd_hyperparam_val_sorted.iloc[0,:]  #experimented with the 3 index, this one acutally performs better

In [ ]:
(size_set, nodes_1, nodes_2, nodes_3, learning_rate, dropout_rate,
 r2_val, recommended_nr_epochs) = pd_hyperparam_val_sorted.iloc[0,:] 
nodes_1 = int(nodes_1)
nodes_2 = int(nodes_2)
recommended_nr_epochs = int(recommended_nr_epochs)
repeats = 8
if size_set == 'large':
    X_train_temp = X_train
    X_test_temp = X_test
    n_features = X_train_temp.shape[1]
elif size_set == 'small':
    X_train_temp = X_train_small
    X_test_temp = X_test
    n_features = X_train_temp.shape[1]
else:
    raise ValueError('size_set must be small or large')
if np.isnan(nodes_3):
    ann = models.Sequential(
        [
        layers.Input(shape = (n_features,)),
        layers.Dense(nodes_1, activation = "relu", name = 'layer1'),
        layers.Dropout(dropout_rate),
        layers.Dense(nodes_2, activation = "relu", name = 'layer2'),
        layers.Dropout(dropout_rate),
        layers.Dense(1, activation = "linear", name = 'layer3'),
        ]
    )
else:  
    nodes_3 = int(nodes_3)
    ann = models.Sequential(
        [
        layers.Input(shape = (n_features,)),
        layers.Dense(nodes_1, activation = "relu", name = 'layer1'),
        layers.Dropout(dropout_rate),
        layers.Dense(nodes_2, activation = "relu", name = 'layer2'),
        layers.Dropout(dropout_rate),
        layers.Dense(nodes_3, activation = "relu", name ='layer3'),
        layers.Dropout(dropout_rate),
        layers.Dense(1, activation = "linear", name = 'layer4'),
        ]
    )
ann.summary()
mean_r2_train, mean_r2_test, out_dict, models_list =full_training_loop(ann, repeats,
    X_train_temp, X_test_temp,y_train,y_test,Cstar, exec_training,
    recommended_nr_epochs,pad,learning_rate = learning_rate, verbose = 0)


In [ ]:
ensemble_plot(Cstar, X_full.index, out_dict)

In [ ]:
recommended_nr_epochs

In [ ]:
print('test')
print(out_dict['r2_test_list'])
print(np.min(out_dict['r2_test_list']))
print(np.max(out_dict['r2_test_list']))

print('train')
print(out_dict['r2_train_list'])
print(np.min(out_dict['r2_train_list']))
print(np.max(out_dict['r2_train_list']))

In [ ]:
# import keras_tuner as kt
# def build_model(hp):
#     """
#     Builds model and sets up hyperparameter space to search.
    
#     Parameters
#     ----------
#     hp : HyperParameter object
#         Configures hyperparameters to tune.
        
#     Returns
#     -------
#     model : keras model
#         Compiled model with hyperparameters to tune.
#     """
#     # Initialize sequential API and start building model.
#     model = models.Sequential()
#     model.add(layers.Input(shape = (n_features,)))
    
#     # Number of Units: 4 - 28 with stepsize of 8
#     #add a first layer
#     model.add(layers.Dense(
#         units = hp.Int("units_1", min_value =4, max_value = 28, step = 8),
#         activation='relu'
#     ))
#     # Tune dropout layer with values from 0 - 0.4 with stepsize of 0.2
#     dropout_rate = hp.Float("dropout", 0, 0.4, step=0.2)
#     model.add(layers.Dropout(dropout_rate))
#     for i in range(hp.Int("num_layers", 2, 4)): 
#         model.add(
#             layers.Dense(
#                 units=hp.Int("units_" + str(i), min_value=4, max_value=28, step=8),
#                 activation="relu")
#             )
#         model.add(
#             layers.Dropout(dropout_rate)
#         )
#     # Add output layer.
#     model.add(layers.Dense(units=1, activation='linear'))
    
#     # Tune learning rate for Adam optimizer with values from 0.01, 0.001, or 0.0001
#     #hp_learning_rate = hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4])
    
#     # Define optimizer, loss, and metrics
#     model.compile(optimizer=keras.optimizers.Adam(),#learning_rate=hp_learning_rate),
#                   loss=keras.losses.MeanSquaredError(),
#                   metrics = tfa.metrics.RSquare()
#                   )
    
#     return model

In [ ]:
# tuner = kt.GridSearch(build_model,
#                       objective= kt.Objective("val_r_square", direction="max"),
#                       project_name = 'data/ml_obs_op_data/ann/GridSearch_full',
#                       executions_per_trial = repeats)

In [ ]:
# tuner.search(X_train, y_train, epochs = 100, batch_size = 32,
#                validation_split = validation_split)

In [ ]:
[a for a in range(2,3)]